In [1]:
# pip install rasterio

In [1]:
import os
import numpy as np
import torch
from torchvision import transforms
import pandas as pd
from math import floor ,log10
from torch import nn, optim
from tqdm.autonotebook import tqdm
from sklearn.metrics import accuracy_score
from torch.utils.data import DataLoader, RandomSampler
from torch.utils.tensorboard import SummaryWriter
import argparse
from sklearn.metrics import roc_curve, confusion_matrix
import warnings
import torch.nn.functional as nnf
import matplotlib.pyplot as plt
import seaborn as sns
import rasterio as rio
import time 
from torchvision.models import resnet50
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv('/hpc/home/srs108/history.csv')

In [3]:
df

,epoch,train_loss,val_loss,train_iou,val_iou,train_acc,val_acc
0,1,1.190840,1.133607,0.056460,0.043494,0.968877,0.974746
1,2,1.164949,1.119090,0.056205,0.044043,0.969616,0.974746
2,3,1.128882,1.075146,0.054092,0.040056,0.969139,0.974746
3,4,1.094905,1.027226,0.040842,0.022111,0.967611,0.974746
4,5,1.054504,0.990963,0.013751,0.003282,0.967917,0.974746
5,6,1.024019,0.966123,0.001248,0.000124,0.968186,0.974746
6,7,1.013201,0.949408,0.000055,0.000019,0.967312,0.974746
7,8,0.994900,0.938894,0.000009,NaN,0.968209,0.974746
8,9,0.989056,0.932630,0.000000,NaN,0.968117,0.974746
9,10,0.991728,0.928633,NaN,NaN,0.967380,0.974746


In [3]:
torch.manual_seed(1000)

In [4]:
class SmokePlumeDataset():
    """SmokePlume Dataset class.

    The image data directory is expected to contain two directories, one labeled
    `positive` and one labeled `negative`, containing the corresponding image
    files.

    The function `create_dataset` can be used as a wrapper to create a
    data set.

    :param datadir: (str) image directory root, has to contain `negative` and
                    `positive` subdirectories, required
    :param mult: (int) factor by which to multiply data set size, default=1
    :param transform: (`torchvision.transform` object) transformations to be
                      applied, default: `None`
    :param balance: (str) method for balancing the data set; `'upsample'` the
                    smaller of the two classes or `'downsample'` the larger
                    of the two. Anything else omits balancing.
    """
    
    def __init__(self, datadir=None, mult=1, transform=None, balance='upsample'):
        self.datadir = datadir
        self.transform = transform
        self.imgfiles = []  # list of image files
        self.labels = []    # list of image file labels
        self.positive_indices = []  # list of indices for positive examples
        self.negative_indices = []  # list of indices for negative examples

        # read in image file names
        idx = 0
        for root, dirs, files in os.walk(datadir):
            for filename in files:
                if not filename.endswith('.jpg'):
                    continue
                self.imgfiles.append(os.path.join(root, filename))
                if 'positive' in root:
                    self.labels.append(True)
                    self.positive_indices.append(idx)
                    idx += 1
                elif 'negative' in root:
                    self.labels.append(False)
                    self.negative_indices.append(idx)
                    idx += 1

        # turn lists into arrays
        self.imgfiles = np.array(self.imgfiles)
        self.labels = np.array(self.labels)
        self.positive_indices = np.array(self.positive_indices)
        self.negative_indices = np.array(self.negative_indices)

        # balance sample, if desired
        if balance == 'downsample':
            self.balance_downsample()
        elif balance == 'upsample':
            self.balance_upsample()

        # increase data set size by factor `mult`
        if mult > 1:
            self.imgfiles = np.array([*self.imgfiles] * mult)
            self.labels = np.array([*self.labels] * mult)
            self.positive_indices = np.array([*self.positive_indices] * mult)
            self.negative_indices = np.array([*self.negative_indices] * mult)

    def __len__(self):
        """Returns length of data set."""
        return len(self.imgfiles)

    def balance_downsample(self):
        """Balance data set by downsampling the negative class (which is the larger of the two)."""
        subsample_idc = np.ravel([
            self.negative_indices,
            self.positive_indices[
                np.random.randint(0, len(self.positive_indices),
                                  len(self.negative_indices))]]).astype(int)

        # adjust other class attributes accordingly
        self.imgfiles = self.imgfiles[subsample_idc]
        self.labels = self.labels[subsample_idc]
        self.positive_indices = np.arange(0, len(self.labels), 1)[
            self.labels == True]
        self.negative_indices = np.arange(0, len(self.labels), 1)[
            self.labels == False]

    def balance_upsample(self):
        """Balance data set by upsampling the positive class (which is the smaller of the two)."""
        subsample_idc = np.ravel([
            self.negative_indices[
                np.random.randint(0, len(self.negative_indices),
                                  len(self.positive_indices)-
                                  len(self.negative_indices))]]).astype(int)

        # adjust other class attributes accordingly
        self.imgfiles = np.concatenate((self.imgfiles,
                                        self.imgfiles[subsample_idc]), axis=0)
        self.labels = np.concatenate((self.labels,
                                      self.labels[subsample_idc]),
                                     axis=0)

        self.positive_indices = np.arange(0, len(self.labels), 1)[
            self.labels == True]
        self.negative_indices = np.arange(0, len(self.labels), 1)[
            self.labels == False]

    def __getitem__(self, idx):
        """Read in image data, preprocess, and apply transformations."""
        # read in data file
        imgfile = rio.open(self.imgfiles[idx])
        imgdata = np.array([imgfile.read(i) for i in [1,2,3]])

        sample = {
            'idx': idx,
            'img': imgdata,
            'lbl': self.labels[idx],
            'imgfile': self.imgfiles[idx]
        }

        return sample

    def display(self, idx, offset=0.2, scaling=1.5):
        """Display a given example from the data set with index `idx`.

        Only RGB channels are displayed.

        Args:
            idx (int): Image index to be displayed.
            offset (float): Constant scaling offset (on a range [0,1]).
            scaling (float): Scaling factor.

        Returns:
            `matplotlib.pyplot.figure` object.
        """
        imgdata = self[idx]['img']

        # scale image data
        imgdata = offset + scaling * (
            np.dstack([imgdata[3], imgdata[2], imgdata[1]]) -
            np.min([imgdata[3], imgdata[2], imgdata[1]])) / \
                (np.max([imgdata[3], imgdata[2], imgdata[1]]) -
                 np.min([imgdata[3], imgdata[2], imgdata[1]]))

        f, ax = plt.subplots(1, 1, figsize=(3, 3))
        ax.imshow((imgdata - np.min(imgdata, axis=(0, 1))) /
                  (np.max(imgdata, axis=(0, 1)) -
                   np.min(imgdata, axis=(0, 1))))

        return f

class ToTensor:
    """Convert ndarrays in sample to Tensors."""
    def __call__(self, sample):
        """Convert sample to Tensor.

        Args:
            sample (dict): Sample to be converted to Tensor.

        Returns:
            dict: Converted Tensor sample.
        """
        out = {'idx': sample['idx'],
               'img': torch.from_numpy(sample['img'].copy()),
               'lbl': sample['lbl'],
               'imgfile': sample['imgfile']}
        return out

class Normalize:
    """Normalize pixel values to zero mean and range [-1, +1] measured in
    standard deviations."""

    def __init__(self):
        self.channel_means = np.array(
            [809.2, 900.5, 1061.4, 1091.7, 1384.5, 1917.8,
             2105.2, 2186.3, 2224.8, 2346.8, 1901.2, 1460.42])
        self.channel_stds = np.array(
            [441.8, 624.7, 640.8, 718.1, 669.1, 767.5,
             843.3, 947.9, 882.4, 813.7, 716.9, 674.8])

    def __call__(self, sample):
        """Normalize the given sample.

        Args:
            sample (dict): Sample to be normalized.

        Returns:
            dict: Normalized sample.
        """
        sample['img'] = (sample['img'] - self.channel_means.reshape(
            sample['img'].shape[0], 1, 1)) / self.channel_stds.reshape(
            sample['img'].shape[0], 1, 1)

        return sample


class Randomize:
    """Randomize image orientation including rotations by integer multiples of
       90 deg, (horizontal) mirroring, and (vertical) flipping."""

    def __call__(self, sample):
        """Randomly rotate, mirror, and/or flip the given sample.

        Args:
            sample (dict): Sample to be randomized.

        Returns:
            dict: Randomized sample.
        """
        imgdata = sample['img']

        # mirror horizontally
        mirror = np.random.randint(0, 2)
        if mirror:
            imgdata = np.flip(imgdata, 2)
        # flip vertically
        flip = np.random.randint(0, 2)
        if flip:
            imgdata = np.flip(imgdata, 1)
        # rotate by [0,1,2,3]*90 deg
        rot = np.random.randint(0, 4)
        imgdata = np.rot90(imgdata, rot, axes=(1, 2))

        return {'idx': sample['idx'],
                'img': imgdata.copy(),
                'lbl': sample['lbl'],
                'imgfile': sample['imgfile']}


class RandomCrop:
    """Randomly crop 90x90 pixel image (from 120x120)."""

    def __call__(self, sample):
        """Randomly crop a 90x90 pixel region from the given sample.

        Args:
            sample (dict): Sample to be cropped.

        Returns:
            dict: Randomized sample.
        """
        imgdata = sample['img']

        x, y = np.random.randint(0, 30, 2)

        return {'idx': sample['idx'],
                'img': imgdata.copy()[:, y:y+90, x:x+90],
                'lbl': sample['lbl'],
                'imgfile': sample['imgfile']}


def create_dataset(*args, apply_transforms=True, **kwargs):
    """Create a dataset with the given arguments and transformations.

    Args:
        *args: Arguments for SmokePlumeDataset.
        apply_transforms (bool): If True, apply available transformations.
        **kwargs: Keyword arguments for SmokePlumeDataset.

    Returns:
        SmokePlumeDataset: Data set.
    """
    if apply_transforms:
        data_transforms = transforms.Compose([
            Normalize(),
            Randomize(),
            ToTensor(),
        ])
    else:
        data_transforms = None

    data = SmokePlumeDataset(*args, **kwargs, transform=data_transforms)

    return data


In [5]:
# device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.resnet_pretrained = resnet50(pretrained=True)
        self.resnet_pretrained.conv1 = nn.Conv2d(3, 64, kernel_size=(3, 3),
                              stride=(2, 2),padding=(3, 3), bias=False)
        
        self.fc1 = nn.Linear(self.resnet_pretrained.fc.out_features, 256)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 1)

    def forward(self, image):
        img_features = self.resnet_pretrained(image)
        img_features = torch.flatten(img_features, 1)
        img_features = self.fc1(img_features)
        x = self.relu(img_features)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        return x

model = CNN()

In [6]:
batch_size = 16
# Create datasets
data_train = create_dataset(datadir="../train_jpg", mult=1)
data_test = create_dataset(datadir="../test_jpg", mult=1)

# Initialize data loaders
train_dl = DataLoader(data_train, batch_size=batch_size, num_workers=4, pin_memory=True)
test_dl = DataLoader(data_test, batch_size=batch_size)

In [7]:
def train(model, opt, loss, epoch):
    model.train()
    train_loss_total, train_acc_total = 0, 0
    # train_score_total, train_label_total, train_pred_total = [], [], []
    progress = tqdm(enumerate(train_dl), desc="Train Loss: ", total=len(train_dl))
    
    for i, batch in progress:
        x = batch['img'].float().to(device)
        y = batch['lbl'].float().to(device)
        output = model(x)

        # Derive binary output
        output_binary = np.zeros(output.shape)
        output_binary[output.cpu().detach().numpy() >= 0] = 1

        # Derive accuracy score
        label = y.cpu().detach().numpy()
        acc = accuracy_score(label, output_binary)
        train_acc_total += acc

        # Calculate loss
        loss_epoch = loss(output, y.reshape(-1, 1))
        train_loss_total += loss_epoch.item()
        progress.set_description("Train Loss: {:.4f}".format(train_loss_total / (i + 1)))

        # Learning
        opt.zero_grad()
        loss_epoch.backward()
        opt.step()
    return train_loss_total, train_acc_total, i 

In [8]:
def test(model, loss, opt, epoch):
    model.eval()

    test_loss_total, test_acc_total = 0, 0
    test_label_total, test_pred_total = [], []
    progress = tqdm(enumerate(test_dl), total=len(test_dl))

    for k, batch in progress:
        x, y = batch['img'].float().to(device), batch['lbl'].float().to(device)
        output = model(x)

        # Calculate loss
        loss_epoch = loss(output, y.reshape(-1, 1))
        test_loss_total += loss_epoch.item()
        progress.set_description("Test Loss: {:.4f}".format(test_loss_total / (k + 1)))

        # Derive binary output
        output_binary = np.zeros(output.shape)
        output_binary[output.cpu().detach().numpy() >= 0] = 1
        label = y.cpu().detach().numpy()

        test_pred_total += list(output_binary[:, 0])
        test_label_total += list(label)

        # Derive accuracy score
        acc = accuracy_score(y.cpu().detach().numpy(), output_binary)
        test_acc_total += acc
    return test_loss_total, test_acc_total, test_label_total, test_pred_total, k

In [9]:
def train_model(model, epochs, opt, loss, batch_size):
    """Wrapper function for model training.

    :param model: model instance
    :param epochs: (int) number of epochs to be trained
    :param opt: optimizer instance
    :param loss: loss function instance
    :param batch_size: (int) batch size"""
    
    loss_train, loss_test, train_acc,test_acc =[],[],[],[]
    epoch_table = pd.DataFrame()
    # Create datasets
    data_train = create_dataset(datadir="../train_jpg", mult=1)
    data_test = create_dataset(datadir="../test_jpg", mult=1)

    # Initialize data loaders
    train_dl = DataLoader(data_train, batch_size=batch_size, num_workers=4, pin_memory=True)
    test_dl = DataLoader(data_test, batch_size=batch_size)

    # Start training process
    test_loss_total_list, test_acc_total_list = [], []
    epoch_list = []
    for epoch in range(epochs):
        epoch_list.append(epoch)
        #Training 
        train_loss_total, train_acc_total, i = train(model, opt, loss, epoch)
            
        # Logging
        writer.add_scalar("training loss", train_loss_total / (i + 1), epoch)
        writer.add_scalar("training acc", train_acc_total / (i + 1), epoch)
        writer.add_scalar('learning_rate', opt.param_groups[0]['lr'], epoch)
        test_loss_total_list.append(train_loss_total / (i + 1))
        test_acc_total_list.append(train_acc_total / (i + 1))
        torch.cuda.empty_cache()
        
        # Test
        test_loss_total, test_acc_total, test_label_total, test_pred_total, k = test(model, loss, opt, epoch)
        
        # Logging
        writer.add_scalar("test loss", test_loss_total / (k + 1), epoch)
        writer.add_scalar("test accuracy", test_acc_total / (k + 1), epoch)
        test_loss_total_list.append(test_loss_total / (k + 1))
        test_acc_total_list.append(test_acc_total / (k + 1))

        # Generate confusion matrix
        if epoch == epochs - 1:
            array = confusion_matrix(np.array(test_label_total), np.array(test_pred_total))
            cm = pd.DataFrame(array, range(2), range(2))
            svm = sn.heatmap(cm, annot=True)
            figure = svm.get_figure()
            image_dir = "../images/test"
            figure.savefig(f"{image_dir}/conf_matrix.png")

        # Screen output
        print(("Epoch {:d}: train loss={:.3f}, test loss= {:.3f}, "
               "train acc={:.3f}, test acc={:.3f}").format(
                   epoch + 1, train_loss_total / (i + 1), test_loss_total / (k + 1),
                   train_acc_total / (i + 1), test_acc_total / (k + 1)))

        loss_train.append(train_loss_total / (i + 1))
        loss_test.append(test_loss_total / (k + 1))
        train_acc.append(train_acc_total / (i + 1))
        test_acc.append(test_acc_total / (k + 1))

        torch.cuda.empty_cache()

        
        
    epoch_table['epoch'] = epoch_list
    epoch_table["training loss"] = train_loss_total_list
    epoch_table["training accuracy"] = train_acc_total_list
    epoch_table["test loss"] = test_loss_total_list
    epoch_table["test accuracy"] = test_acc_total_list

    return model, epoch_table

In [10]:
# setup argument parser
parser = argparse.ArgumentParser()
parser.add_argument('-f')
parser.add_argument('-ep', type=int, default=100, help='Number of epochs')
parser.add_argument('-bs', type=int, default=16, help='Batch size')
parser.add_argument('-lr', type=float, default=0.003, help='Learning rate')
parser.add_argument('-mo', type=float, default=0.7, help='Momentum')
args = parser.parse_args()

# initialize tensorboard writer
writer = SummaryWriter('runs/' + "ep{}_lr{:.0e}_bs{:03d}_mo{:.1f}/".format(
    args.ep, args.lr, args.bs, args.mo))

# initialize loss, optimizer, and scheduler
loss = nn.BCEWithLogitsLoss()
if torch.cuda.is_available():
    model.cuda()
optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=args.mo)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min',
                                                factor=0.5, threshold=1e-4,
                                                min_lr=1e-6)

In [12]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# run model training
start = time.time()
trained_model, epoch_table = train_model(model, args.ep, optimizer, loss, args.bs)
end = time.time()
print("Time taken:", end - start)
# writer.close()

In [18]:
# Set 'epoch' column as index
epoch_table = epoch_table.set_index("epoch")

# Convert training and test accuracy values to percentages
epoch_table['training accuracy'] = epoch_table['training accuracy']*100
epoch_table['test accuracy'] = epoch_table['test accuracy']*100

# Round all values in the DataFrame to two decimal places
epoch_table = epoch_table.apply(lambda x: round(x, 2))

# Visualize the table
epoch_table.style.background_gradient(cmap='Blues').set_precision(2)

In [ ]:
# Plot training and testing accuracy
epoch_table_acc = epoch_table[['training accuracy','test accuracy']]
acc_plot = sns.relplot(data=epoch_table_acc, kind="line")
acc_plot.set(ylabel="accuracy",title = "Training and testing accuracy")
plt.savefig("accuracy.png")

In [ ]:
# Plot training and testing loss
epoch_table_loss = epoch_table[['training loss','test loss']]
acc_plot = sns.relplot(data=epoch_table_loss, kind="line")
acc_plot.set(ylabel="loss",title = "Training and testing loss")
plt.savefig("loss.png")